In [1]:
import pandas as pd
import numpy as np
import json

filename = 'skill_builder_data_corrected.csv'
df = pd.read_csv(filename, encoding='ISO-8859-1', low_memory=False)
df = df[(df['original'] == 1) & (df['attempt_count'] == 1) & ~(df['skill_name'].isnull())]

In [3]:
df.shape

(240503, 30)

In [2]:
df.head()

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0,NaN,1,1
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0,NaN,2,2
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0,NaN,2,2
5,35450555,220674,70363,33172,51457,1,1,1,16031,tutor,...,0,4,16031,30060,NaN,12,0,NaN,4,4
6,35450573,220674,70363,33174,51459,1,1,1,15047,tutor,...,0,4,15047,30060,NaN,6,0,NaN,5,5


In [6]:
def generate_datasets():
    users_list = df['user_id'].unique()
    skill_list = df['skill_name'].unique()
    
    skill_dict = dict(zip(skill_list, np.arange(len(skill_list), dtype='int32') + 1))
    response_list = []
    skill_list = []
    assistment_list = []
    
    counter = 0
    for user in users_list:
        sub_df = df[df['user_id'] == user]
        if len(sub_df) > 100:
            first_hundred = sub_df.iloc[0:100]
            response_df = pd.DataFrame(index=[counter], columns=['student_id']+['r'+str(i) for i in range(100)])
            skill_df = pd.DataFrame(index=[counter], columns=['student_id']+['s'+str(i) for i in range(100)])
            assistment_df = pd.DataFrame(index=[counter], columns=['student_id']+['a'+str(i) for i in range(100)])
            
            response_df.iloc[0, 0] = first_hundred.iloc[0]['user_id']
            skill_df.iloc[0, 0] = first_hundred.iloc[0]['user_id']
            assistment_df.iloc[0, 0] = first_hundred.iloc[0]['user_id']
            for i in range(100):
                response_df.iloc[0, i+1] = first_hundred.iloc[i]['correct']
                skill_df.iloc[0, i+1] = skill_dict[first_hundred.iloc[i]['skill_name']]
                assistment_df.iloc[0, i+1] = first_hundred.iloc[i]['assistment_id']
            counter += 1
            response_list.append(response_df)
            skill_list.append(skill_df)
            assistment_list.append(assistment_df)
    
    response_df = pd.concat(response_list)
    skill_df = pd.concat(skill_list)
    assistment_df = pd.concat(assistment_list)
    
    return skill_dict, response_df, skill_df, assistment_df
    
skill_dict, response_df, skill_df, assistment_df = generate_datasets()

with open('skill_dict.json', 'w', encoding='utf-8') as f:
    to_dump_dict = {}
    for key, value in skill_dict.items():
        to_dump_dict[key] = str(value)
    json.dump(to_dump_dict, f)
response_df.to_csv('correct.tsv', sep='\t')
skill_df.to_csv('skill.tsv', sep='\t')
assistment_df.to_csv('assistment_id.tsv', sep='\t')
print('Done')

Done


In [7]:
response_df = pd.read_csv('correct.tsv', sep='\t').drop('Unnamed: 0', axis=1)
skill_df = pd.read_csv('skill.tsv', sep='\t').drop('Unnamed: 0', axis=1)
assistment_df = pd.read_csv('assistment_id.tsv', sep='\t').drop('Unnamed: 0', axis=1)
skill_dict = {}
with open('skill_dict.json', 'r', encoding='utf-8') as f:
    loaded = json.load(f)
    for k, v in loaded.items():
        skill_dict[k] = int(v)

skill_num = len(skill_dict) + 1 # including 0

def one_hot(skill_matrix, vocab_size):
    '''
    params:
        skill_matrix: 2-D matrix (student, skills)
        vocal_size: size of the vocabulary
    returns:
        a ndarray with a shape like (student, sequence_len, vocab_size)
    '''
    seq_len = skill_matrix.shape[1]
    result = np.zeros((skill_matrix.shape[0], seq_len, vocab_size))
    for i in range(skill_matrix.shape[0]):
        result[i, np.arange(seq_len), skill_matrix[i]] = 1.
    return result

def dkt_one_hot(skill_matrix, response_matrix, vocab_size):
    seq_len = skill_matrix.shape[1]
    skill_response_array = np.zeros((skill_matrix.shape[0], seq_len, 2 * vocab_size))
    masking_array = np.zeros((skill_matrix.shape[0], seq_len, 2 * vocab_size))
    for i in range(skill_matrix.shape[0]):
        skill_response_array[i, np.arange(seq_len), 2 * skill_matrix[i] + response_matrix[i]] = 1.
        masking_array[i, np.arange(seq_len), 2 * skill_matrix[i]] = 1.
        masking_array[i, np.arange(seq_len), 2 * skill_matrix[i] + 1] = 1.
    return skill_response_array, masking_array

def preprocess(skill_df, response_df, skill_num):
    skill_matrix = skill_df.iloc[:, 1:].values
    response_array = response_df.iloc[:, 1:].values
    skill_array = one_hot(skill_matrix, skill_num)
    skill_response_array, masking_array = dkt_one_hot(skill_matrix, response_array, skill_num)
    return skill_array, response_array, skill_response_array, masking_array
    

skill_array, response_array, skill_response_array, masking_array = preprocess(skill_df, response_df, skill_num)

In [9]:
!pip install keras

    100% |████████████████████████████████| 276kB 1.2MB/s ta 0:00:01


In [8]:
import keras
from keras.layers import Input, Dense, LSTM, TimeDistributed, Lambda, multiply
from keras.models import Model
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

def build_skill2skill_model(input_shape, lstm_dim=32, dropout=0.0):
    input = Input(shape=input_shape, name='input skills')
    lstm = LSTM(lstm_dim, 
                return_sequences=True, 
                dropout=dropout,
                name='lstm layer')(input)
    output = TimeDistributed(Dense(input_shape[-1], activation='softmax'), name='probability')(lstm)
    model = Model(inputs=[input], outputs=[output])
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def reduce_dim(x):
    x = K.max(x, axis=-1, keepdims=True)
    return x

def build_dkt_model(input_shape, lstm_dim=32, dropout=0.0):
    input_skills = Input(shape=input_shape, name='input skills')
    lstm = LSTM(lstm_dim, 
                return_sequences=True, 
                dropout=dropout,
                name='lstm layer')(input_skills)
    dense = TimeDistributed(Dense(input_shape[-1], activation='sigmoid'), name='probability for each')(lstm)
    
    skill_next = Input(shape=input_shape, name='next_skill_tested')
    merged = multiply([dense, skill_next], name='multiply')
    reduced = Lambda(reduce_dim, output_shape=(input_shape[0], 1), name='reduce dim')(merged)
    
    model = Model(inputs=[input_skills, skill_next], outputs=[reduced])
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

print('skill2skill')
skill2skill_model = build_skill2skill_model((99, skill_num), lstm_dim=64)

print('dkt')
dkt_model = build_dkt_model((99, 2 * skill_num), lstm_dim=64)
    

ImportError: No module named 'keras'

In [26]:
# train skill2skill
skill2skill_model.fit(skill_array[:, 0:-1], 
                      skill_array[:, 1:],
                      epochs=20, 
                      batch_size=32, 
                      shuffle=True,
                      validation_split=0.2)

Train on 467 samples, validate on 117 samples
Epoch 1/20
467/467 [==============================] - 1s - loss: 4.6219 - acc: 0.2211 - val_loss: 4.6031 - val_acc: 0.1231
Epoch 2/20
467/467 [==============================] - 1s - loss: 4.2417 - acc: 0.1989 - val_loss: 4.4726 - val_acc: 0.0401
Epoch 3/20
467/467 [==============================] - 1s - loss: 3.6987 - acc: 0.1516 - val_loss: 4.0881 - val_acc: 0.1029
Epoch 4/20
467/467 [==============================] - 1s - loss: 3.2477 - acc: 0.2674 - val_loss: 3.7969 - val_acc: 0.1197
Epoch 5/20
467/467 [==============================] - 1s - loss: 2.8260 - acc: 0.3376 - val_loss: 3.5260 - val_acc: 0.1874
Epoch 6/20
467/467 [==============================] - 1s - loss: 2.4923 - acc: 0.4107 - val_loss: 3.2665 - val_acc: 0.2543
Epoch 7/20
467/467 [==============================] - 1s - loss: 2.2316 - acc: 0.4820 - val_loss: 3.0331 - val_acc: 0.3516
Epoch 8/20
467/467 [==============================] - 1s - loss: 2.0180 - acc: 0.5395 - val_l

In [27]:
dkt_model.fit([skill_response_array[:, 0:-1], masking_array[:, 1:]],
              response_array[:, 1:, np.newaxis],
              epochs=20, 
              batch_size=32, 
              shuffle=True,
              validation_split=0.2)

Train on 467 samples, validate on 117 samples
Epoch 1/20
467/467 [==============================] - 1s - loss: 0.6695 - acc: 0.8080 - val_loss: 0.6539 - val_acc: 0.8330
Epoch 2/20
467/467 [==============================] - 1s - loss: 0.5896 - acc: 0.8450 - val_loss: 0.5370 - val_acc: 0.8351
Epoch 3/20
467/467 [==============================] - 1s - loss: 0.4594 - acc: 0.8481 - val_loss: 0.4614 - val_acc: 0.8342
Epoch 4/20
467/467 [==============================] - 1s - loss: 0.4107 - acc: 0.8487 - val_loss: 0.4549 - val_acc: 0.8336
Epoch 5/20
467/467 [==============================] - 1s - loss: 0.3954 - acc: 0.8488 - val_loss: 0.4469 - val_acc: 0.8342
Epoch 6/20
467/467 [==============================] - 1s - loss: 0.3888 - acc: 0.8498 - val_loss: 0.4451 - val_acc: 0.8352
Epoch 7/20
467/467 [==============================] - 1s - loss: 0.3853 - acc: 0.8510 - val_loss: 0.4473 - val_acc: 0.8358
Epoch 8/20
467/467 [==============================] - 1s - loss: 0.3821 - acc: 0.8505 - val_l